In [14]:

from typing import SupportsIndex, Optional

import clingo
import clingo.ast
from clingo.ast import ProgramBuilder

In [15]:
program = """

head1(X,Y,Z) :- body1(X), body2(Y), body3(Z).

head2(X) :- body1(X), body2(X-1).

head3(X) :- body4(X).

"""

In [16]:
def get_parsed_program(program: str):
    nodes = []
    clingo.ast.parse_string(program, lambda stm: nodes.append(stm))
    return nodes

In [17]:
nodes = get_parsed_program(program)
nodes

[ast.Program(Location(begin=Position(filename='<string>', line=1, column=1), end=Position(filename='<string>', line=1, column=1)), 'base', []),
 ast.Rule(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=46)), ast.Literal(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=13)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=13)), 'head1', [ast.Variable(Location(begin=Position(filename='<string>', line=3, column=7), end=Position(filename='<string>', line=3, column=8)), 'X'), ast.Variable(Location(begin=Position(filename='<string>', line=3, column=9), end=Position(filename='<string>', line=3, column=10)), 'Y'), ast.Variable(Location(begin=Position(filename='<string>', line=3, column=11), end=Position(filename='<string>', line=3, column=12)), 'Z')], 0))), [ast.Literal(

In [18]:
pos = clingo.ast.Position('<string>', 0, 0)
loc = clingo.ast.Location(pos, pos)

In [ ]:
def _body_insert_node(rule: clingo.ast.AST, node: clingo.ast.AST, index: Optional[SupportsIndex] = None):
    head = rule.head
    body = list(rule.body)
    __index = index or len(body)
    body.insert(__index, node)
    return clingo.ast.Rule(loc, head, body)

In [19]:
def _body_insert_pos_symbol(rule: clingo.ast.AST, literal: clingo.Symbol, index: Optional[SupportsIndex] = None):
    return _body_insert_node(rule,
                             clingo.ast.Literal(loc, clingo.ast.Sign.NoSign,
                                                clingo.ast.SymbolicAtom(clingo.ast.SymbolicTerm(loc, literal))),
                             index)

In [21]:
def _body_insert_neg_symbol(rule: clingo.ast.AST, literal: clingo.Symbol, index: Optional[SupportsIndex] = None):
    return _body_insert_node(rule,
                             clingo.ast.Literal(loc, clingo.ast.Sign.Negation,
                                                clingo.ast.SymbolicAtom(clingo.ast.SymbolicTerm(loc, literal))),
                             index)


In [20]:
def _body_delete_literal(rule: clingo.ast.AST, index: SupportsIndex = -1):
    head = rule.head
    body = list(rule.body)
    body.pop(index)
    return clingo.ast.Rule(loc, head, body)


In [22]:
rule = nodes[1]
print(rule)

head1(X,Y,Z) :- body1(X); body2(Y); body3(Z).


In [23]:
new_rule_1 = _body_insert_pos_symbol(rule, clingo.Function('body4', [clingo.Number(0)]))
print(new_rule_1)

head1(X,Y,Z) :- body1(X); body2(Y); body3(Z); body4(0).


In [24]:
new_rule_2 = _body_insert_neg_symbol(new_rule_1, clingo.Function('missing', [
    clingo.Function('value', [clingo.Function('one'), clingo.Function('two')])]), 2)
print(new_rule_2)

head1(X,Y,Z) :- body1(X); body2(Y); not missing(value(one,two)); body3(Z); body4(0).


In [25]:
new_rule_3 = _body_delete_literal(new_rule_2, -2)
print(new_rule_3)

head1(X,Y,Z) :- body1(X); body2(Y); not missing(value(one,two)); body4(0).


In [26]:
new_rule_4 = _body_delete_literal(new_rule_3)
print(new_rule_4)

head1(X,Y,Z) :- body1(X); body2(Y); not missing(value(one,two)).


In [27]:
new_rule_5 = _body_delete_literal(new_rule_4)
print(new_rule_5)

head1(X,Y,Z) :- body1(X); body2(Y).


In [ ]:
ctl = clingo.Control()
with ProgramBuilder(ctl) as pb:
    pb.add(new_rule_1)
    pb.add(new_rule_2)
    pb.add(new_rule_3)
    pb.add(new_rule_4)
    pb.add(new_rule_5)